# This notebook can be executed after getRegionalBox.ipynb

In [ ]:
using Pkg
using Metal

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv"  # maybe GeoPoints and planet1D should be fusioned

# batchGPU should be at this level (I have not made it as a module yet, since the choice of Metal/CUDA should be done in a manual way)
include("../src/batchFiles/batchGPU.jl")


include("../src/commonBatchs.jl")
include("../src/flexOPT.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

using .commonBatchs, .flexOPT, .planet1D, .GeoPoints

# give me a model of material variables (like seismic models)

In [ ]:
# what you need as a model:

boxGrids = lazyProduceOrLoad("MarmousiCoordInfo"); # for a big model, I do not recommend to store these data

seismicModelMarmousi=lazyProduceOrLoad("seismicModelMarmousi");

# choose your physics and get the semi symbolic OPT expression(s) 

In [ ]:
famousEquationType="2DacousticTime"
@show myEquationInside=famousEquations(famousEquationType)


In [ ]:
Δnum = (1.0,1.0,1.0) # this should be in the same order as myEquationInside.coordinates 

# or
 
Δnum = (boxGrids.Δx,boxGrids.Δz,1.0) # hahaha this Δnum is not very good since there is no reason for the grids to be rectangular with this isotropic medium but let's try


# hereafter are the parameters which are important for modellers but maybe not interesting (so default numbers are given)

In [ ]:
# concerning the order of test B-spline functions (-1 denotes a pulse function)
orderBtime=1
orderBspace=1
# the number of neighbouring points used to construct operators (except the point itself)
pointsInSpace=2
pointsInTime=2
# in order to control higher order partials, we need the supplemantary order > 0, for an ill-posed inversion of Taylor coefficients
supplementaryOrder=2
# B-spline order for the interpolated Taylor expansion
WorderBspace=1
WorderBtime=1

TaylorOptions=(WorderBtime=WorderBtime,WorderBspace=WorderBspace,supplementaryOrder=supplementaryOrder)
trialFunctionsCharacteristics=(orderBtime=orderBtime,orderBspace=orderBspace,pointsInSpace=pointsInSpace,pointsInTime=pointsInTime)



In [ ]:
OPTcoefConfig = @strdict myEquationInside=myEquationInside Δnum=Δnum TaylorOptions=TaylorOptions trialFunctionsCharacteristics=trialFunctionsCharacteristics 
coefs = myProduceOrLoad(OPTobj,OPTcoefConfig,"OPTcoefs") #attention! without GPU, OPTobj will take a lot of time (for >2D)

In [ ]:
@show size(coefs["output"].AjiννᶜUs)
#@show coefs["output"].ΓjkννᶜFs

# now we construct numerical operators based on the semi-symbolic rules that we constructed above

In [ ]:
models=[] # you might need to make this empty tuple first, otherwise one-member tuple can be misinterpreted
models=push!(models,seismicModelMarmousi.Vpv.*1.e3) 
 


# in this example, the material variable is just v and we use Vpv 

# put fake Nt here for quasi-numerical operator construction


In [ ]:
models=push!(models,seismicModelMarmousi.Vpv.*1.e3) 
modelPoints = getModelPoints(models[1],pointsInTime,coefs["output"].utilities.timeMarching)#reference model dimension (this is important to explicitly define since the dimension/size of material variable array(s) can differ from the model domain)

In [ ]:
IneedExternalSources = true
maskedRegionForSourcesInSpace = nothing
# if IneedExternalSources and if the source region is localised in space then
maskedRegionForSourcesInSpace  = Array{CartesianIndex,1}(undef,0) # it is important to decalre the type of this
maskedRegionForSourcesInSpace = push!(maskedRegionForSourcesInSpace, CartesianIndex(modelPoints[1:end-1].÷2))# in Ndimension (or Ndimension  - 1 if timeMarching)
# in this example, I put a point source at the centre of the model space

forceModels =((1.0)) # if your model does not have anything special material parameters then it's how it's written

In [ ]:
include("../src/flexOPT.jl")

In [ ]:
lhsConfigurations = @strdict semiSymbolicCoefs=coefs models modelName="MarmousiVp" modelPoints maskedRegion=nothing
numOperators = myProduceOrLoad(constructingNumericalDiscretisedEquations,lhsConfigurations,"numOperators","lhs")